In [2]:
import web3
import json
import array
import math
import random
import time
import csv
import numpy as np

from web3 import Web3, IPCProvider, HTTPProvider

In [101]:
web3 = Web3(HTTPProvider("HTTP://127.0.0.1:7545", request_kwargs = {'timeout':600}))
print(web3)

In [3]:
web3.eth.accounts

['0xC56fC154A17f382B258Ca73Cd976A4c87593457C',
 '0x068D68C9e07289B4b750f0fFad4214bF52579F73',
 '0x20cb29F38c76166ECa7Cab4e25B2845c4a9502D7',
 '0x9e890CB220237fF0d8687C342d8B62115489FEaA',
 '0xf4ad0182f7B9E1bdd6E43faCd0f8A9e2Fe53E50B',
 '0x91B98A2a9F8dB7e831422510Af9eEcC8544Bf97E',
 '0x244748d929DD43366d3601BAFAD984329FE7aEc5',
 '0xb6e182138632067599d008c0e7f1ac2E5168430c',
 '0x15b09B2A5Df2E648a16a126107A0553e550da0ca',
 '0x219d5B205B85d89d085595f8c3d6cf687C3E72e7']

In [4]:
web3.eth.getBalance(web3.eth.accounts[0])

100000000000000000000

In [35]:
web3.personal.unlockAccount(web3.eth.accounts[0],'gogogo')

True

In [6]:
coinbase = web3.eth.coinbase
coinbase

'0xC56fC154A17f382B258Ca73Cd976A4c87593457C'

In [100]:
contract_address = web3.toChecksumAddress(0xb9f1cb717ebcb5e3200254231ea768ad65151788)
abi_str = '[{"constant":false,"inputs":[{"name":"_timepoint","type":"uint8"},{"name":"_schedulePower","type":"uint16"},{"name":"_NegPower","type":"uint16"},{"name":"_PosPower","type":"uint16"},{"name":"_NegEnergy","type":"uint16"},{"name":"_Posenergy","type":"uint16"},{"name":"_Negprice","type":"uint16"},{"name":"_PosPrice","type":"uint16"}],"name":"Setflexoffer","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]'
contract_abi = json.loads(abi_str)
contract = web3.eth.contract(abi=contract_abi, address=contract_address)
contract

In [8]:
def csv_import(filename):

    columns = []
    with open(filename, 'rU') as f:
        reader = csv.reader(f, delimiter=';')
        for row in reader:
            if columns:
                for i, value in enumerate(row):
                    columns[i].append(value)
            else:
                # first row
                columns = [[value] for value in row]
    # you now have a column-major 2D array of your file.
    as_dict = {c[0]: c[1:] for c in columns}
    return as_dict

In [102]:
data = csv_import('Flexoffer.csv')
data

E:\it\envs\py3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 'U' mode is deprecated
  after removing the cwd from sys.path.


In [103]:
int(data['Leistung_Plan'][21])


20000

In [107]:
tx_hashes = [None]*96

In [108]:
contract.transact({'from':web3.eth.accounts[0]}).Setflexoffer(5,
                                                       int(data['Leistung_Plan'][5]),
                                                       int(data['Leistung-'][5]),
                                                       int(data['Leistung+'][5]),
                                                       int(data['Energie-'][5]),
                                                       int(data['Energie+'][5]),
                                                       int(data['Preis-'][5]),
                                                       int(data['Preis+'][5]))

HexBytes('0xd849c438a48d7366f33ef87f92a534c3297b81ddc5ccc965c54a5f867bb40a06')

In [109]:
for i in range(0,95):
    #web3.personal.unlockAccount(web3.eth.accounts[0],'gogogo')
    tx_hash = contract.transact({'from':web3.eth.accounts[0]}).Setflexoffer(i,
                                                                       int(data['Leistung_Plan'][i]),
                                                                       int(data['Leistung-'][i]),
                                                                       int(data['Leistung+'][i]),
                                                                       int(data['Energie-'][i]),
                                                                       int(data['Energie+'][i]),
                                                                       int(data['Preis-'][i]),
                                                                       int(data['Preis+'][i]))
    tx_hashes[i] = tx_hash.hex()

In [97]:

#print(tx_hashes)

In [87]:
re=web3.eth.waitForTransactionReceipt(tx_hashes[6])
re

AttributeDict({'transactionHash': HexBytes('0x9a8c9289900ede6b524fa84dab7354d1c217d54f16ecd8846970b2c35be46048'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xa6cc4fe97575f3c9af75ed5ac901456faf48be7dbae95394f9b6e38eb1989b66'),
 'blockNumber': 306,
 'gasUsed': 65726,
 'cumulativeGasUsed': 65726,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')})

In [88]:
re.gasUsed

65726

In [89]:
gasUsed = 0
for i in range(0,95):
        receipt = web3.eth.waitForTransactionReceipt(tx_hashes[i])
        gasUsed += receipt.gasUsed

In [90]:
gasUsed

6265474

In [93]:
for ii in range(18,75): 
    tx_hash = contract.transact({'from':web3.eth.accounts[0]}).Setflexoffer(i,
                                                                       int(data['Leistung_Plan'][i]),
                                                                       int(data['Leistung-'][i]),
                                                                       int(data['Leistung+'][i]),
                                                                       int(data['Energie-'][i]),
                                                                       int(data['Energie+'][i]),
                                                                       int(data['Preis-'][i]),
                                                                       int(data['Preis+'][i]))
    tx_hashes[i] = tx_hash.hex()

In [ ]:
for ii in range(18,75): 
    tx_hash = contract.transact({'from':web3.eth.accounts[0]}).Setflexoffer(i,
                                                                       int(data['Leistung_Plan'][i]),
                                                                       int(data['Leistung-'][i]),
                                                                       int(data['Leistung+'][i]),
                                                                       int(data['Energie-'][i]),
                                                                       int(data['Energie+'][i]),
                                                                       int(data['Preis-'][i]),
                                                                       int(data['Preis+'][i]))
    tx_hashes[i] = tx_hash.hex()

In [95]:
gasUsed1 = 0
for i in range(18,75):
        receipt = web3.eth.waitForTransactionReceipt(tx_hashes[i])
        gasUsed1 += receipt.gasUsed

gasUsed1

3767502